# [CSI-30] Atividade 4  - Predictive analytics

##### Importação de Bibliotecas

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pymssql
import csv
import time
import math
import warnings

from sklearn.linear_model import LinearRegression
from datetime import datetime, timedelta

In [2]:
warnings.filterwarnings('ignore')

In [3]:
# pd.set_option('display.max_rows', None)

## Criação dos SGBD (MSSQL Server 2017)

In [4]:
!docker-compose -f ../docker-compose.yml up -d

/bin/bash: /home/lucas/anaconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
[+] Running 1/0
 ✔ Network csi-30-at4_default        Creat...                              0.0s 
 ⠋ Container csi-30-at4-sqlserver-1  Creating                              0.0s 
[+] Running 2/2
 ✔ Network csi-30-at4_default        Creat...                              0.0s 
 ✔ Container csi-30-at4-sqlserver-1  Created                               0.1s 
[+] Running 2/2
 ✔ Network csi-30-at4_default        Creat...                              0.0s 
 ✔ Container csi-30-at4-sqlserver-1  Created                               0.1s 
[+] Running 2/2
 ✔ Network csi-30-at4_default        Creat...                              0.0s 
 ✔ Container csi-30-at4-sqlserver-1  Created                               0.1s 
[+] Running 2/2
 ✔ Network csi-30-at4_default        Creat...                              0.0s 
 ✔ Container csi-30-at4-sqlserver-1  Created                               0.1s 
[

## Conexão com o MSSQL Server 2017

#### Aguarde o container inicializar!

In [5]:
time.sleep(10)

In [6]:
conn = pymssql.connect(host='localhost', user='sa', password='Pent@gon1979', database='master')

In [7]:
conn.autocommit(True)

In [8]:
cursor = conn.cursor()

### Modelo Dimensional

![Diagrama](../images/schema_dim.png)

### Cria da tabela e relacionamentos do Banco de dados Dimensional

In [9]:
cursor.execute("CREATE DATABASE FI_DW;")

In [10]:
with open('../sql/ddl_dimensional.sql', 'r') as file:
    ddl_create_dimensional = file.read()

In [11]:
commands = [command.strip() for command in ddl_create_dimensional.split('GO') if command.strip()]

In [12]:
try:
    for command in commands:
        cursor.execute(command)
    
except Exception as e:
    print(f"Erro: {str(e)}")

## Extraction

##### Informações Diárias

In [13]:
path_files_data = [
                                                                                    202111, 202112,
    202201, 202202, 202203, 202204, 202205, 202206, 202207, 202208, 202209, 202210, 202211, 202212,
    202301, 202302, 202303, 202304, 202305, 202306, 202307, 202308, 202309, 202310  
]

In [14]:
inf_diario = pd.DataFrame()

In [15]:
for data in path_files_data:
    new_df = f"../data/inf_diario/inf_diario_fi_{data}.csv"
    inf_diario = pd.concat([inf_diario, pd.read_csv(new_df, sep=';')], axis=0)

##### Informações cadastrais

In [16]:
cad = pd.read_csv('../data/cad/cad_fi.csv', sep=';', encoding='iso-8859-1', low_memory=False)

## Transformation and Load

#### Dimension Controller

![fact_Sales](../images/dim_Controller.png)

In [17]:
def create_dim_Controller(conn, cursor, cad):
    try:
        for controller in cad['CONTROLADOR'].unique():
            temp = cad[cad['CONTROLADOR'] == controller]
            if len(temp) == 0:
                continue

            CNPJController = temp.iloc[0, 1]
            command = ("INSERT INTO dim_Controller (NameController, CNPJController) VALUES (%s, %s)")
            cursor.execute(command, (controller, CNPJController))    
            conn.commit()


    except Exception as e:
        print(f"Erro: {str(e)}")
        conn.rollback()

In [18]:
create_dim_Controller(conn, cursor, cad)

#### Dimension Auditor

![fact_Sales](../images/dim_Auditor.png)

In [19]:
def create_dim_Auditor(conn, cursor, cad):
    try:
        for auditor in cad['AUDITOR'].unique():
            temp = cad[cad['AUDITOR'] == auditor]
            if len(temp) == 0:
                continue

            CNPJAuditor = temp.iloc[0, 1]
            command = ("INSERT INTO dim_Auditor (NameAuditor, CNPJAuditor) VALUES (%s, %s)")
            cursor.execute(command, (auditor, CNPJAuditor))    
            conn.commit()
        
    except Exception as e:
        print(f"Erro: {str(e)}")
        conn.rollback()

In [20]:
create_dim_Auditor(conn, cursor, cad)

#### Dimension Custodian

![fact_Sales](../images/dim_Custodian.png)

In [21]:
def create_dim_Custodian(conn, cursor, cad):
    try:
        for custodian in cad['CUSTODIANTE'].unique():
            temp = cad[cad['CUSTODIANTE'] == custodian]
            if len(temp) == 0:
                continue

            CNPJCustodian = temp.iloc[0, 1]
            command = ("INSERT INTO dim_Custodian (NameCustodian, CNPJCustodian) VALUES (%s, %s)")
            # print(custodian, CNPJCustodian)
            cursor.execute(command, (custodian, CNPJCustodian))    
            conn.commit()
    
    except Exception as e:
        print(f"Erro: {str(e)}")
        conn.rollback()

In [22]:
create_dim_Custodian(conn, cursor, cad)

#### Dimension Admin

![fact_Sales](../images/dim_Admin.png)

In [23]:
def create_dim_Admin(conn, cursor, cad):
    try:
        for admin in cad['ADMIN'].unique():
            temp = cad[cad['ADMIN'] == admin]
            if len(temp) == 0:
                continue

            CNPJAdmin = temp.iloc[0, 1]
            command = ("INSERT INTO dim_Admin (NameAdmin, CNPJAdmin) VALUES (%s, %s)")
            # print(admin, CNPJAdmin)
            cursor.execute(command, (admin, CNPJAdmin))    
            conn.commit()
            
    except Exception as e:
        print(f"Erro: {str(e)}")
        conn.rollback()

In [24]:
create_dim_Admin(conn, cursor, cad)

#### Dimension Manager

![fact_Sales](../images/dim_Manager.png)

In [25]:
def create_dim_Manager(conn, cursor, cad):
    try:
        for manager in cad['GESTOR'].unique():
            temp = cad[cad['GESTOR'] == manager][['PF_PJ_GESTOR', 'CPF_CNPJ_GESTOR']]
            if len(temp) == 0:
                continue

            PfPjGestor = temp.iloc[0, 0]
            CpfCnpjManager = temp.iloc[0, 1]

            command = ("INSERT INTO dim_Manager (CPF_CNPJManager, NameJManager, PF_PJ_Manager) VALUES (%s, %s, %s)")
            cursor.execute(command, (CpfCnpjManager, manager, PfPjGestor))    
            conn.commit()
        
    except Exception as e:
        print(f"Erro: {str(e)}")
        conn.rollback()

In [26]:
create_dim_Manager(conn, cursor, cad)

#### Dimension Investiment Fund

![fact_Sales](../images/dim_Investment_Fund.png)

In [31]:
def tratar_nan(valor):
    return None if valor == 'nan' or (isinstance(valor, (float, int)) and math.isnan(valor)) else valor

In [34]:
def create_dim_Investment_Fund(conn, cursor, cad):
    for cnpj in cad['CNPJ_FUNDO'].unique():
        try:

            temp = cad[cad['CNPJ_FUNDO'] == cnpj][['DENOM_SOCIAL', 'CNPJ_FUNDO', 'CLASSE_ANBIMA', 'DT_REG', 'DT_CONST',
                                                   'CD_CVM', 'SIT', 'TAXA_PERFM', 'TAXA_ADM', 'DIRETOR', 'FUNDO_COTAS',
                                                   'FUNDO_EXCLUSIVO', 'CLASSE', 'CONDOM', 'RENTAB_FUNDO', 'TP_FUNDO',
                                                   'PUBLICO_ALVO']]
                
            for index, row in temp.iterrows():

                DenomSocial = tratar_nan(row[0])
                CNPJ = tratar_nan(row[1])
                ClassAMBIMA = tratar_nan(row[2])
                DataReg = tratar_nan(row[3])
                DataConst = tratar_nan(row[4])
                CodCVM = tratar_nan(row[5])
                Situation = tratar_nan(row[6])
                TaxPerfomance = tratar_nan(row[7])
                TaxAdm = tratar_nan(row[8])
                Director = tratar_nan(row[9])
                FundCotas = tratar_nan(row[10])
                FundExclusive = tratar_nan(row[11])
                Classe = tratar_nan(row[12])
                Condom = tratar_nan(row[13])
                RentabFund = tratar_nan(row[14])
                TypeFund = tratar_nan(row[15])
                TargetAudience = tratar_nan(row[16])

                if FundCotas is not None:
                    FundCotas = 1 if FundCotas == 'S' else 0

                if FundExclusive is not None:
                    FundExclusive = 1 if FundExclusive == 'S' else 0

                command = ("""
                    INSERT INTO dim_Investment_Fund (
                        DenomSocial, 
                        CNPJ, 
                        ClassAMBIMA, 
                        DataReg, 
                        DataConst, 
                        CodCVM, 
                        Situation, 
                        TaxPerfomance, 
                        TaxAdm, 
                        Director, 
                        FundCotas, 
                        FundExclusive, 
                        Classe, 
                        Condom, 
                        RentabFund, 
                        TypeFund, 
                        TargetAudience)
                    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                """
                )

                cursor.execute(command, (
                    DenomSocial, 
                    CNPJ, 
                    ClassAMBIMA, 
                    DataReg, 
                    DataConst, 
                    CodCVM, 
                    Situation, 
                    TaxPerfomance, 
                    TaxAdm, 
                    Director, 
                    FundCotas, 
                    FundExclusive, 
                    Classe, 
                    Condom, 
                    RentabFund, 
                    TypeFund, 
                    TargetAudience
                ))

                conn.commit()
            
        except Exception as e:
            print(f"Erro: {str(e)}")
            conn.rollback()

In [35]:
create_dim_Investment_Fund(conn, cursor, cad)

#### Dimension Time

![fact_Sales](../images/dim_Time.png)

In [36]:
def create_dim_Time(conn, cursor, datetime, day, week, month, quarter, year):

    try:
        command = ("INSERT INTO dim_Time ( \
                    datetime, day, week, month, quarter, year) \
                    VALUES (%s, %s, %s, %s, %s, %s)")

        cursor.execute(command, (datetime, day, week, month, quarter, year))

        conn.commit()

    except Exception as e:
        print(f"Erro: {str(e)}")
        conn.rollback()

In [37]:
auxTime = set()

In [38]:
for _, row in inf_diario.iterrows():
    auxTime.add(row['DT_COMPTC'])

In [39]:
auxTimeList = sorted(list(auxTime))

In [40]:
for date_string in auxTimeList:
 
    dt = datetime.strptime(date_string, '%Y-%m-%d')

    day = dt.day
    week = dt.strftime('%U')
    quarter = math.ceil(dt.month / 4)
    year = dt.year
    month = dt.month   

    create_dim_Time(
        conn,
        cursor,
        date_string,
        day,
        week,
        month,
        quarter,
        year
    )

In [41]:
del auxTime
del auxTimeList

#### Fact Value

![fact_Sales](../images/fact_Value.png)

In [ ]:
def getSurrogateIDbyOriginalID(cursor, table, nameID, nameSurrogateID, key):
    cursor.execute(f'SELECT {nameSurrogateID} FROM {table} WHERE {nameID} = {key}')
    
    return cursor.fetchall()[0][0]

In [ ]:
def create_fact_Sales_register(conn, cursor, id_dim_employee, id_dim_customer, id_dim_territory,
                               id_dim_salesOrderHeader, id_dim_time, id_dim_product, SalesOrderDetailID,
                               ProductQty, UnitPrice, LineTotal):

    try:
        command = """
            INSERT INTO fact_SalesItem (
                id_dim_employee, id_dim_customer, id_dim_territory, id_dim_salesOrderHeader,
                id_dim_time, id_dim_product, SalesOrderDetailID, ProductQty, UnitPrice, LineTotal)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """

        values = [id_dim_employee, id_dim_customer, id_dim_territory, id_dim_salesOrderHeader, id_dim_time, id_dim_product, SalesOrderDetailID, ProductQty, UnitPrice, LineTotal]
        values = [None if v == 'None' or v == 'nan' or (isinstance(v, float) and math.isnan(v)) else v for v in values]

        cursor.execute(command, tuple(values))

        conn.commit()

    except Exception as e:
        print(f"Erro: {str(e)}")
        conn.rollback()


## Data analytics

In [ ]:
query1 = '''
    SELECT 
        e.DepartmentName AS Departamento,                
        t.year AS Ano,                   
        SUM(f.LineTotal) AS 'Total Vendas' 
    FROM 
        fact_SalesItem f                 
    JOIN 
        dim_Employee e ON f.id_dim_employee = e.id_dim_employee  
    JOIN 
        dim_Time t ON f.id_dim_time = t.id_dim_time              
    GROUP BY 
        e.DepartmentName, t.year        
    ORDER BY 
        e.DepartmentName, t.year;
'''


df1 = pd.read_sql(query1, conn).set_index(['Departamento', 'Ano'])
df1['Total Vendas'] = df1['Total Vendas'].apply(lambda x: f"${x:,.2f}")


df1

In [ ]:
inf_diario.dropna(subset=['DT_COMPTC'], axis=0, inplace=True)

In [ ]:
cad = cad[cad['TP_FUNDO'] == 'FI']

In [ ]:
situation = cad[['DENOM_SOCIAL', 'CNPJ_FUNDO', 'SIT', 'VL_PATRIM_LIQ']][cad['SIT'] == 'EM FUNCIONAMENTO NORMAL']
situation = situation[situation['VL_PATRIM_LIQ'] >= 100000000]
situation.drop('VL_PATRIM_LIQ', axis=1, inplace=True)

In [ ]:
situation.sample(3)

In [ ]:
data = pd.merge(situation, inf_diario, how='inner', on='CNPJ_FUNDO').set_index(['DT_COMPTC'])

In [ ]:
data.drop(['CNPJ_FUNDO', 'SIT'], axis=1, inplace=True)

In [ ]:
data = data.pivot_table(values='VL_PATRIM_LIQ', index='DT_COMPTC', columns='DENOM_SOCIAL')
data.index = pd.to_datetime(data.index)

In [ ]:
data.head(5)

In [ ]:
data = data.loc[:, ~data.iloc[0, :].isnull()]

In [ ]:
data = data.loc[:, ~data.iloc[-1, :].isnull()]

In [ ]:
plt.figure(figsize=(10, 6))

for column in np.random.choice(data.columns, size=5, replace=False):
    sns.lineplot(data=data, x=data.index, y=data[column]/1000000000, label = column)

plt.xlabel('Data')
plt.ylabel('Bilhão de Reais')
plt.xticks(rotation=90)

plt.legend(title='Legenda', loc='upper center', bbox_to_anchor=(0.5, -0.3), fancybox=True, shadow=True, ncol=1)

# Ajustando o layout para evitar sobreposições
plt.tight_layout()


# plt.legend()
plt.show()

In [ ]:
CoefficientOfVariation = pd.DataFrame(columns=['beta_mean', 'beta_std', 'Coefficient_of_variation'])

In [ ]:
predictions = dict()
coef = dict()
indexes = dict()

In [ ]:
for column, serie in data.items():
    beta = serie / serie.shift(1) - 1    
    new_data = pd.DataFrame({'beta_mean': [beta.mean()], 'beta_std': [beta.std()], 'Coefficient_of_variation': [beta.mean() / beta.std()]}, 
                        index=[column]) 

    CoefficientOfVariation = pd.concat([CoefficientOfVariation, new_data])
    
    X = []
    y = []

    for index, b in enumerate(beta):
        if(not (np.isnan(b) or np.isinf(b))):
            X.append(index)
            y.append(b)

    X = np.array(X).reshape(-1, 1)
    y = np.array(y).reshape(-1, 1)

    model = LinearRegression()
    model.fit(X, y)

    predictions[column] = model.predict(np.array([i for i in range(735)]).reshape(-1, 1))
    coef[column] = model.coef_

In [ ]:
CoefficientOfVariation = CoefficientOfVariation.loc[~CoefficientOfVariation['Coefficient_of_variation'].isna(), :]

In [ ]:
CoefficientOfVariation.sample(3)

In [ ]:
cdi_daily = 0.00050788 

In [ ]:
filter = (CoefficientOfVariation['Coefficient_of_variation'] >= 0.4) & \
         (CoefficientOfVariation['Coefficient_of_variation'] <= 2) & \
         (CoefficientOfVariation['beta_mean'] >= cdi_daily)

In [ ]:
noBest = CoefficientOfVariation[~filter].sort_values(by='beta_mean', ascending=False)

In [ ]:
best = CoefficientOfVariation[filter].sort_values(by='beta_mean', ascending=False)

In [ ]:
plt.figure(figsize=(10, 6))

ax = sns.scatterplot(data=best, x="beta_std", y="beta_mean", marker='+')
ax = sns.scatterplot(data=noBest, x="beta_std", y="beta_mean", marker='+')

ax.set_ylim(0, 0.0175)
ax.set_xlim(0, 0.06)

ax.axhline(y=cdi_daily, linestyle='--', linewidth=2, color='r')
ax.text(x=0.05, y=0.0009, s='CDI', color='r', va='center', ha='right')

ax.text(x=0.006, y=0.009, s='2', color='gray', va='center', ha='right')
ax.text(x=0.027, y=0.009, s='0.4', color='gray', va='center', ha='right')
ax.text(x=0.02, y=0.012, s='Melhores FI', color='g', va='center', ha='right')

plt.plot([0, 0.00875], [0, 0.0175], linestyle='--', linewidth=1, color='gray')
plt.plot([0, 0.04375], [0, 0.0175], linestyle='--', linewidth=1, color='gray')

plt.xlabel('Risco (Desvio padrão do retorno diário/Beta)')
plt.ylabel('Retorno (Média do retorno diário/Beta)')


plt.title('Coeficiente de Variação (Retorno/Risco)')
plt.tight_layout()

plt.show()

In [ ]:
best.sample(3)

In [ ]:
plt.figure(figsize=(10, 10))

for index, row in best.head(1).iterrows():
    datas_str = data.index.copy()
    datas_lista = datas_str.to_list()
    ultima_data = datas_lista[-1]

    novas_datas = [ultima_data + timedelta(days=i) for i in range(1, 246)]

    datas_lista.extend(novas_datas)
    novo_index = pd.Index(datas_lista)
    
    sns.lineplot(data=data, x=data.index, y=index, label = index)
#     sns.lineplot(x=novo_index, y=predictions[index].reshape(-1), label = index)
    

plt.xlabel('Data')
plt.ylabel('Centena de Milhão de Reais')
plt.xticks(rotation=90)

plt.legend(title='Legenda', loc='upper center', bbox_to_anchor=(0.5, -0.2), fancybox=True, shadow=True, ncol=1)


# plt.legend()
plt.show()

x = np.array([i for i in range(735)])
y = predictions[index].reshape(-1)


sns.lineplot(x=x, y=y, label = index)
plt.show()

In [ ]:
index = 'VALORA PREV XP SEGUROS FIC DE FUNDOS DE INVESTIMENTO RENDA FIXA CRÉDITO PRIVADO'

In [ ]:
datas_str = data.index.copy()
datas_lista = datas_str.to_list()
ultima_data = datas_lista[-1]

novas_datas = [ultima_data + timedelta(days=i) for i in range(1, 246)]

datas_lista.extend(novas_datas)
novo_index = pd.Index(datas_lista)

sns.lineplot(x=novo_index, y=predictions[index].reshape(-1), label = index)

In [ ]:
predictions[index].reshape(-1)

In [ ]:
predictions[index]

In [ ]:
# MSSQL Server
conn.close()